In [167]:
import pandas as pd
import json
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [177]:
df = pd.read_json('dwts_scraper/scores.json')
df['dance_id'] = np.arange(df.shape[0])

In [178]:
df['judge'] = df['judge_phrase'].str.split(",", expand = False)

In [179]:
# Some weeks (e.g., random have a `score` column instead of a `scores` column.  Rename that.

# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_20)#Weekly_scores Week 5.
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_17)#Weekly_scores Week 6.


# https://stackoverflow.com/questions/34989341/how-to-remove-nan-value-while-combining-two-column-in-panda-data-frame
df['scores'] = df['scores'].fillna(df['score'])

In [180]:
df.columns

Index(['couple', 'scores', 'dance', 'music', 'result', 'season', 'week',
       'judge_phrase', 'tv_show', 'original_couple', 'score', 'film_theme',
       'disney_film', 'film', 'technical_score', 'performance_score',
       'dance_chosen_by', 'couple_trio_dance_partner_',
       'cirque_du_soleil_show', 'couple_professionals_', 'broadway_show',
       'dances_chosen_by', 'stevie_wonder_music', 'team_captain', 'era',
       'cher_music', 'movie_genre', 'couple_judge_', 'villain', 'dynamic_duo',
       'results', 'dance_theme_chosen_by', 'couple_team_up_judge_',
       'guest_performers', 'music_iconic_routine_', 'icon', 'musical', 'date',
       'performers', 'tracks_performed', 'dancers', 'britney_spears_music',
       'tribute_to_', 'grease_music', 'horror_film_show', 'queen_music',
       'unnamed_5', 'janet_jackson_music', 'dance_id', 'judge'],
      dtype='object')

In [156]:
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_9)#Weekly_scores Week 6 has a 'ranked order' score dance.
# Dance Offs are also a problem, some are separate tables, some are bottom rows in tables.






In [157]:
# Some weeks have a technical score/performance score. but no scores column.

# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_11)#Weekly_scores Week 4
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_10)#Weekly_scores Week 4

# double_score = df[df['technical_score'].notna()]

# double_score[double_score.columns[~double_score.isnull().any()]]

In [181]:
# Quite a few rows from tables that shouldn't be parsed (after end of week shows).  e.g. tributes.
# Either fix in parsing or drop here.

df.dropna(subset=['dance'], inplace=True)

# df[df['dance'].isna()]



In [182]:
# Some couple columns have additional info which is in parens in the column. This creates an NA couple column.

pattern = r'(?P<couple_alone>.*?)\((?P<additional>.*?)\)$'

# e.g., https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_27) Week 4.  Couple (Trio Dance Partner)
df[['couple_alone','trio_partner']] = df['couple_trio_dance_partner_'].str.extract(pattern)
df['couple'] = df['couple'].fillna(df['couple_alone'])
df.drop(columns='couple_alone', inplace=True)

# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_16) Week 5 (Professionals).
df[['couple_alone','additional_dancers']] = df['couple_professionals_'].str.extract(pattern)
df['couple'] = df['couple'].fillna(df['couple_alone'])
df.drop(columns='couple_alone', inplace=True)

# couple_team_up_judge_ from https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_22)#Week_8:_Judges'_Team-up_Challenge
# Some of These are multi-couple dances, with a judge listed i the couple column, and then an X for the score that the judge that coached the team would have given.
# but others are regular dances (just a different column name)
df['couple'] = df['couple'].fillna(df['couple_team_up_judge_'])

# couple_judge_  from https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_11)#Week_10:_Finals is much simpler (just a judge chosen dance).
df[['couple_alone','judge_choosing_dance']] = df['couple_judge_'].str.extract(pattern)
df['couple'] = df['couple'].fillna(df['couple_alone'])
df.drop(columns='couple_alone', inplace=True)

# One remains from the couple_judge_ due to a substitution which then didn't have a judge (so the regex above doesn't catch it)
# 2341 where the couple_judge_ is Nastia & Sasha[a]
df['couple'] = df['couple'].fillna(df['couple_judge_'])
# df.iloc[2341]

df.drop(columns=['couple_trio_dance_partner_', 'couple_professionals_', 'couple_judge_'], inplace=True)

# df[df['couple'].isna()]

In [214]:
# Some have technical_score and performance_score.
# add a score_type column.
# split these out, going to be two rows, each with scores column and a score_type column.

double_scores = df[df['technical_score'].notna()].drop(columns="scores")
# now get two rows per
double_scores.melt(id_vars = 'dance_id', value_vars = ['technical_score','performance_score'], value_name = "scores", var_name = "score_type")

single_scores = df.drop(df.index[df['technical_score'].notna()])
# make others be score_type
single_scores['score_type'] = "single_score"
single_scores.drop(columns=['technical_score','performance_score'], inplace=True)

# This is not working because I don't know how merge works.  Need the rest of the columns from double_scores I think.
df = double_scores.merge(single_scores, how = "left", on = "dance_id")

In [215]:
df_types.head(100)

,couple_x,dance_x,music_x,result_x,season_x,week_x,judge_phrase_x,tv_show_x,original_couple_x,score_x,film_theme_x,disney_film_x,film_x,technical_score_x,performance_score_x,dance_chosen_by_x,cirque_du_soleil_show_x,broadway_show_x,dances_chosen_by_x,stevie_wonder_music_x,team_captain_x,era_x,cher_music_x,movie_genre_x,villain_x,dynamic_duo_x,results_x,dance_theme_chosen_by_x,couple_team_up_judge__x,guest_performers_x,music_iconic_routine__x,icon_x,musical_x,date_x,performers_x,tracks_performed_x,dancers_x,britney_spears_music_x,tribute_to__x,grease_music_x,horror_film_show_x,queen_music_x,unnamed_5_x,janet_jackson_music_x,dance_id,judge_x,trio_partner_x,additional_dancers_x,judge_choosing_dance_x,total_score_x,indiv_scores_x,score_type_x,couple_y,scores,dance_y,music_y,result_y,season_y,week_y,judge_phrase_y,tv_show_y,original_couple_y,score_y,film_theme_y,disney_film_y,film_y,technical_score_y,performance_score_y,dance_chosen_by_y,cirque_du_soleil_show_y,broadway_show_y,dances_chosen_by_y,stevie_wonder_music_y,team_captain_y,era_y,cher_music_y,movie_genre_y,villain_y,dynamic_duo_y,results_y,dance_theme_chosen_by_y,couple_team_up_judge__y,guest_performers_y,music_iconic_routine__y,icon_y,musical_y,date_y,performers_y,tracks_performed_y,dancers_y,britney_spears_music_y,tribute_to__y,grease_music_y,horror_film_show_y,queen_music_y,unnamed_5_y,janet_jackson_music_y,judge_y,trio_partner_y,additional_dancers_y,judge_choosing_dance_y,total_score_y,indiv_scores_y,score_type_y
0,Erin & Maks,Tango,"""Sweet Dreams (Are Made of This)""—Eurythmics",Safe,10,Week 4: Double-score Week,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.",NaN,NaN,NaN,NaN,NaN,NaN,"18 (6, 6, 6)","21 (7, 7, 7)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,774,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio...",NaN,NaN,NaN,NaN,NaN,single_score,Erin & Maks,NaN,Tango,"""Sweet Dreams (Are Made of This)""—Eurythmics",Safe,10,Week 4: Double-score Week,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.",NaN,NaN,NaN,NaN,NaN,NaN,"18 (6, 6, 6)","21 (7, 7, 7)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio...",NaN,NaN,NaN,NaN,NaN,single_score
1,Evan & Anna,Tango,"""Wait a Minute""—The Pussycat Dolls feat. Timba...",Safe,10,Week 4: Double-score Week,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.",NaN,NaN,NaN,NaN,NaN,NaN,"26 (9, 8, 9)","26 (9, 8, 9)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,920,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio...",NaN,NaN,NaN,NaN,NaN,single_score,Evan & Anna,NaN,Tango,"""Wait a Minute""—The Pussycat Dolls feat. Timba...",Safe,10,Week 4: Double-score Week,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.",NaN,NaN,NaN,NaN,NaN,NaN,"26 (9, 8, 9)","26 (9, 8, 9)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio...",NaN,NaN,NaN,NaN,NaN,single_score
2,Niecy & Louis,Rumba,"""Taking Chances""—Celine Dion",Last to be called safe,10,Week 4: Double-score Week,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.",NaN,NaN,NaN,NaN,NaN,NaN,"18 (6, 6, 6)","18 (6, 6, 6)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,921,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio...",NaN,NaN,NaN,NaN,NaN,single_score,Niecy & Louis,NaN,Rumba,"""Taking Chances""—Celine Dion",Last to be called safe,10,Week 4: Double-score Week,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.",NaN,NaN,NaN,NaN,NaN,NaN,"18 (6, 6, 6)","18 (6, 6, 6)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio...",NaN,NaN,NaN,NaN,NaN,single_score
3,Aiden & Edyta,Rumba,"""Live Like We're Dyin

In [191]:
df[['total_score','indiv_scores']] = df['scores'].str.extract(r'(?P<total_score>\d+)\s\((?P<indiv_scores>.*)\)')

# new_cols['judge_score_list'] = new_cols['indiv_scores'].str.split(",")


# df_new = pd.concat([df, new_cols], axis = 1)

In [174]:
# song_cols = df['music'].str.extract(r'"(?P<song_name>.*)"—(?P<song_artist>.*)$')
# song_cols
# df["music"].str.split(" / ") Some music columns have multiples.

,couple,scores,dance,music,result,season,week,judge_phrase,tv_show,original_couple,score,film_theme,disney_film,film,technical_score,performance_score,dance_chosen_by,cirque_du_soleil_show,broadway_show,dances_chosen_by,stevie_wonder_music,team_captain,era,cher_music,movie_genre,villain,dynamic_duo,results,dance_theme_chosen_by,couple_team_up_judge_,guest_performers,music_iconic_routine_,icon,musical,date,performers,tracks_performed,dancers,britney_spears_music,tribute_to_,grease_music,horror_film_show,queen_music,unnamed_5,janet_jackson_music,dance_id,judge,trio_partner,additional_dancers,judge_choosing_dance


In [192]:
df[df['total_score'].isna()]

# most are multi couple dances, with ranking scores.
# a few have a total score, but not an individual score.  e.g., s9w10.

,couple,scores,dance,music,result,season,week,judge_phrase,tv_show,original_couple,score,film_theme,disney_film,film,technical_score,performance_score,dance_chosen_by,cirque_du_soleil_show,broadway_show,dances_chosen_by,stevie_wonder_music,team_captain,era,cher_music,movie_genre,villain,dynamic_duo,results,dance_theme_chosen_by,couple_team_up_judge_,guest_performers,music_iconic_routine_,icon,musical,date,performers,tracks_performed,dancers,britney_spears_music,tribute_to_,grease_music,horror_film_show,queen_music,unnamed_5,janet_jackson_music,dance_id,judge,trio_partner,additional_dancers,judge_choosing_dance,total_score,indiv_scores
137,Chuck & Anna T.---Ashley & Edyta---Donny & Kym...,6---4---10---8,Salsa,"""Get Busy""—Sean Paul","""Get Busy""—Sean Paul",9,Week 1,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio...",NaN,NaN,NaN,NaN,NaN
138,Aaron & Karina---Mark & Lacey---Tom & Cheryl--...,10---8---4---6,Viennese waltz,"""I'm Your Man""—Leonard Cohen","""I'm Your Man""—Leonard Cohen",9,Week 1,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio...",NaN,NaN,NaN,NaN,NaN
433,Diana & Henry,,Cha-cha-cha,"""Move Your Feet""—Junior Senior",Eliminated,18,Week 2: Celebrity's Pick Night,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,433,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonioli]",NaN,NaN,NaN,NaN,NaN
487,Dance-off,Dance-off,Dance-off,Dance-off,Dance-off,8,Week 2,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,487,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio...",NaN,NaN,NaN,NaN,NaN
629,Wynonna & Tony---Victor & Lindsay---D.L. & Che...,,Freestyle,"""The Rockafeller Skank""—Fatboy Slim","""The Rockafeller Skank""—Fatboy Slim",16,Week 3: Prom Night,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,629,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio...",NaN,NaN,NaN,NaN,NaN
774,Erin & Maks,NaN,Tango,"""Sweet Dreams (Are Made of This)""—Eurythmics",Safe,10,Week 4: Double-score Week,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.",NaN,NaN,NaN,NaN,NaN,NaN,"18 (6, 6, 6)","21 (7, 7, 7)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,774,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio...",NaN,NaN,NaN,NaN,NaN
789,Misty & Maks,,Jive,"""Shake It""—Metro Station",Withdrew,7,Week 3,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,789,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio...",NaN,NaN,NaN,NaN,NaN
847,Ray & Cheryl,,Cha-cha-cha,"""Twist and Shout""—The Beatles",Withdrew,28,Week 3: Movie Night,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.",NaN,NaN,NaN,NaN,NaN,Ferris Bueller's Day Off,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,847,"[ Carrie Ann Inaba, Len Goodman, Bruno Tonio...",NaN,NaN,NaN,NaN,NaN
920,Evan & Anna,NaN,Tango,"""Wait a Minute""—The Pussycat Dolls feat. Timba...",Safe,10,Week 4: Double-score Week,"Carrie Ann Inaba, Len Goodman, Bruno Tonioli.",NaN,NaN,NaN,NaN,NaN,NaN,"26 (9, 8, 9)","26 (9, 8, 9)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
# (      
#     df_new.assign(judge_count = lambda df: df.judge.apply(len))
#           .assign(score_count = lambda df: df.judge_score_list.apply(len))
# )

In [147]:
df_final = ( df_new
              #  .head(5)
               .explode(['judge','judge_score_list'])
               .rename(columns={'judge_score_list': 'judge_score'})
               .drop(columns=['indiv_scores','judge_phrase','scores'])
               .assign(judge = lambda df: df.judge.str.strip(" ."))
)

ValueError: columns must have matching element counts

In [ ]:
all_cols = df_final.columns.values.tolist()

front_cols = [ 'dance_id','season','week','couple','dance','music','judge','judge_score','total_score' ]

remaining_cols = [ele for ele in all_cols if ele not in front_cols]

reordered = front_cols + remaining_cols

In [ ]:
df_final = df_final[reordered]

In [ ]:
df_final